In [2]:
import torch
import argparse
import glob
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
from scipy.special import softmax
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from captum.attr import visualization

from transformers import AutoTokenizer

import datasets
from datasets import load_dataset, load_metric 
from datasets import list_datasets, list_metrics

from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification

###mods
import time

In [3]:
from transformers import BertConfig, BertTokenizer, BertModel
config = BertConfig.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased', config=config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# load model   
model = BertForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-imdb").to("cuda")
model.eval()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("fabriceyhc/bert-base-uncased-imdb")

In [6]:
dataset = "imdb" # select a dataset
dataset = load_dataset(dataset) # load dataset

Found cached dataset imdb (/home/anwbw/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
special_tokens = {"[CLS]", "[SEP]"}
special_idxs = {101,102}    
mask = "[PAD]"
mask_id = 0   

In [8]:
def preprocess_sample(text):
    tokenized_input  = tokenizer(text, add_special_tokens=True, truncation=True)
    input_ids = tokenized_input['input_ids']
    text_ids = (torch.tensor([input_ids])).to("cuda")
    text_words = tokenizer.convert_ids_to_tokens(text_ids[0])
    
    # mask special tokens
    # attention_mask: indicates which tokens should be attended to.
    att_mask = tokenized_input['attention_mask']
    spe_idxs = [x for x, y in list(enumerate(input_ids)) if y in special_idxs]
    att_mask = [0 if index in spe_idxs else 1 for index, item in enumerate(att_mask)]
    att_mask = [0 if index in spe_idxs else 1 for index, item in enumerate(att_mask)]
    att_mask = (torch.tensor([att_mask])).to("cuda")
    
    return text_ids, att_mask, text_words

In [9]:
def predict(model, text_ids, target, att_mask=None, seg_ids=None):
    out = model(text_ids, attention_mask=att_mask, token_type_ids=seg_ids)
    prob = out[0]
    pred_class = torch.argmax(prob, axis=1).cpu().detach().numpy()
    pred_class_prob = softmax(prob.cpu().detach().numpy(), axis=1)
    return pred_class[0], pred_class_prob[:, target][0]

In [10]:
#determine which words to be masked::::
#takes: sorted indices of word attributions, text_words, text_ids and numer of words to be masked: replace_num
#returns: truncated_text_ids with top num words removed  
def truncate_words(sorted_idx, text_words, text_ids, replaced_num, seg_ids=None):
    to_be_replaced_idx = []
    i= 0
    while len(to_be_replaced_idx) < replaced_num and i!=len(text_words)-1:
        current_idx = sorted_idx[i]
        if text_words[current_idx] not in special_tokens:
            to_be_replaced_idx.append(current_idx)
        i += 1
    remaining_idx = sorted(list(set(sorted_idx) - set(to_be_replaced_idx)))
    truncated_text_ids = text_ids[0, np.array(remaining_idx)]
    if seg_ids is not None:
        seg_ids = seg_ids[0, np.array(remaining_idx)]
    truncated_text_words = np.array(text_words)[remaining_idx]
    return truncated_text_ids.unsqueeze(0), truncated_text_words, seg_ids

In [11]:
#words to be replaced
##masking k% words::: with mask
#insted of removing (like truncated_words) replaces them with mask
def replace_words(sorted_idx, text_words, text_ids, replaced_num, mask, mask_id):
    to_be_replaced_idx = []
    i= 0
    while len(to_be_replaced_idx) < replaced_num and i!=len(text_words)-1:
        current_idx = sorted_idx[i]
        if text_words[current_idx] not in special_tokens:
            to_be_replaced_idx.append(current_idx)
        i += 1
    replaced_text_ids = text_ids.clone()
    replaced_text_ids[0, to_be_replaced_idx] = mask_id
    replaced_text_words = np.copy(text_words)
    replaced_text_words[to_be_replaced_idx] = mask
    return replaced_text_ids, replaced_text_words

In [12]:
# compute rollout between attention layers
def compute_rollout_attention(all_layer_matrices, start_layer=0):
    
    # adding residual consideration- code adapted from https://github.com/samiraabnar/attention_flow
    num_tokens = all_layer_matrices[0].shape[1]
    batch_size = all_layer_matrices[0].shape[0]
    eye = torch.eye(num_tokens).expand(batch_size, num_tokens, num_tokens).to(all_layer_matrices[0].device)
    all_layer_matrices = [all_layer_matrices[i] + eye for i in range(len(all_layer_matrices))]
    matrices_aug = [all_layer_matrices[i] / all_layer_matrices[i].sum(dim=-1, keepdim=True)
                          for i in range(len(all_layer_matrices))]
    joint_attention = matrices_aug[start_layer]
    for i in range(start_layer+1, len(matrices_aug)):
        joint_attention = matrices_aug[i].bmm(joint_attention)
        
    return joint_attention

In [13]:
class Generator:
    def __init__(self, model):
        self.model = model
        self.model.eval()

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids, attention_mask)

    
    def generate_TransCAM(self, input_ids, attention_mask,
                          index=None, start_layer=0):

        result = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        
        output = result[0]
        hs = result[1]

        kwargs = {"alpha": 1}

        blocks = self.model.bert.encoder.layer

        for blk_id in range(len(blocks)):
            hs[blk_id].retain_grad()

        if index == None:
            index = np.argmax(output.cpu().data.numpy(), axis=-1)

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0, index] = 1
        one_hot_vector = one_hot
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        one_hot = torch.sum(one_hot.cuda() * output)

        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        self.model.relprop(torch.tensor(one_hot_vector).to(input_ids.device), **kwargs)

        cams = {}
        
        for blk_id in range(len(blocks)):
            hs_grads = hs[blk_id].grad
            
            att = blocks[blk_id].attention.self.get_attn().squeeze(0)
            att = att.mean(dim=0)
            att = att.mean(dim=0)
            
            cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
            cat = cat * att
            
            cams[blk_id] = cat
            
        trans_expln = sum(cams.values())

        return trans_expln

    def generate_LRP(self, input_ids, attention_mask,
                     index=None, start_layer=11):
        
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)[0]
        kwargs = {"alpha": 1}

        if index == None:
            index = np.argmax(output.cpu().data.numpy(), axis=-1)

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0, index] = 1
        one_hot_vector = one_hot
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        one_hot = torch.sum(one_hot.cuda() * output)

        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        self.model.relprop(torch.tensor(one_hot_vector).to(input_ids.device), **kwargs)

        cams = []
        blocks = self.model.bert.encoder.layer
        for blk in blocks:
            grad = blk.attention.self.get_attn_gradients()
            cam = blk.attention.self.get_attn_cam()
            cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
            grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
            cam = grad * cam
            cam = cam.clamp(min=0).mean(dim=0)
            cams.append(cam.unsqueeze(0))
        rollout = compute_rollout_attention(cams, start_layer=start_layer)
        rollout[:, 0, 0] = 0
        return rollout[:, 0]

    def generate_LRP_last_layer(self, input_ids, attention_mask,
                     index=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)[0]
        kwargs = {"alpha": 1}
        if index == None:
            index = np.argmax(output.cpu().data.numpy(), axis=-1)

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0, index] = 1
        one_hot_vector = one_hot
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        one_hot = torch.sum(one_hot.cuda() * output)

        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        self.model.relprop(torch.tensor(one_hot_vector).to(input_ids.device), **kwargs)

        cam = self.model.bert.encoder.layer[-1].attention.self.get_attn_cam()[0]
        cam = cam.clamp(min=0).mean(dim=0).unsqueeze(0)
        cam[:, 0, 0] = 0
        return cam[:, 0]

    def generate_full_lrp(self, input_ids, attention_mask,
                     index=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)[0]
        kwargs = {"alpha": 1}

        if index == None:
            index = np.argmax(output.cpu().data.numpy(), axis=-1)

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0, index] = 1
        one_hot_vector = one_hot
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        one_hot = torch.sum(one_hot.cuda() * output)

        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        cam = self.model.relprop(torch.tensor(one_hot_vector).to(input_ids.device), **kwargs)
        cam = cam.sum(dim=2)
        cam[:, 0] = 0
        return cam

    def generate_attn_last_layer(self, input_ids, attention_mask,
                     index=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)[0]
        cam = self.model.bert.encoder.layer[-1].attention.self.get_attn()[0]
        cam = cam.mean(dim=0).unsqueeze(0)
        cam[:, 0, 0] = 0
        return cam[:, 0]

    def generate_rollout(self, input_ids, attention_mask, start_layer=0, index=None):
        self.model.zero_grad()
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)[0]
        blocks = self.model.bert.encoder.layer
        all_layer_attentions = []
        for blk in blocks:
            attn_heads = blk.attention.self.get_attn()
            avg_heads = (attn_heads.sum(dim=1) / attn_heads.shape[1]).detach()
            all_layer_attentions.append(avg_heads)
        rollout = compute_rollout_attention(all_layer_attentions, start_layer=start_layer)
        rollout[:, 0, 0] = 0
        return rollout[:, 0]

    def generate_attn_gradcam(self, input_ids, attention_mask, index=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)[0]
        kwargs = {"alpha": 1}

        if index == None:
            index = np.argmax(output.cpu().data.numpy(), axis=-1)

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0, index] = 1
        one_hot_vector = one_hot
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        one_hot = torch.sum(one_hot.cuda() * output)

        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        self.model.relprop(torch.tensor(one_hot_vector).to(input_ids.device), **kwargs)

        cam = self.model.bert.encoder.layer[-1].attention.self.get_attn()
        grad = self.model.bert.encoder.layer[-1].attention.self.get_attn_gradients()

        cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
        grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
        grad = grad.mean(dim=[1, 2], keepdim=True)
        cam = (cam * grad).mean(0).clamp(min=0).unsqueeze(0)
        cam = (cam - cam.min()) / (cam.max() - cam.min())
        cam[:, 0, 0] = 0
        
        return cam[:, 0]

In [66]:
def generate_explns(explanations, input_ids, attention_mask, start_layer=0, true_class = 1, is_true = True):
    
    if is_true:
        # TransCAM
        TransCAM_expln = explanations.generate_TransCAM(input_ids=input_ids, attention_mask=attention_mask, 
                                                    index=true_class, start_layer=start_layer)
        # LRP
        LRP_expln = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, 
                                                    index=true_class, start_layer=start_layer)[0]
        # PartialLRP
        PartialLRP_expln = explanations.generate_LRP_last_layer(input_ids=input_ids, 
                                                     index=true_class, attention_mask=attention_mask)[0]
        # FullLRP
        # FullLRP_expln = explanations.generate_full_lrp(input_ids=input_ids, attention_mask=attention_mask, 
        #                                           index=true_class)[0]
        # Att
        Att_expln = explanations.generate_attn_last_layer(input_ids=input_ids, attention_mask=attention_mask, 
                                                    index=true_class)[0]
        # Rollout
        Rollout_expln = explanations.generate_rollout(input_ids=input_ids, attention_mask=attention_mask,      
                                                    index=true_class, start_layer=0)[0]
        # Att_Gradcam
        # Att_Gradcam_expln = explanations.generate_attn_gradcam(input_ids=input_ids, attention_mask=attention_mask, 
        #                                            index=true_class)[0]    
        
    else:
        true_class = 1-true_class
        # TransCAM
        TransCAM_expln = explanations.generate_TransCAM(input_ids=input_ids, attention_mask=attention_mask, 
                                                    index=true_class, start_layer=start_layer)
        # LRP
        LRP_expln = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, 
                                                    index=true_class, start_layer=start_layer)[0]
        # PartialLRP
        PartialLRP_expln = explanations.generate_LRP_last_layer(input_ids=input_ids, 
                                                    index=true_class, attention_mask=attention_mask)[0]
        # FullLRP
        # FullLRP_expln = explanations.generate_full_lrp(input_ids=input_ids, attention_mask=attention_mask, 
        #                                            index=true_class)[0]
        # Att
        Att_expln = explanations.generate_attn_last_layer(input_ids=input_ids, attention_mask=attention_mask, 
                                                    index=true_class)[0]
        # Rollout
        Rollout_expln = explanations.generate_rollout(input_ids=input_ids, attention_mask=attention_mask, 
                                                    index=true_class, start_layer=0)[0]
        # Att_Gradcam
        # Att_Gradcam_expln = explanations.generate_attn_gradcam(input_ids=input_ids, attention_mask=attention_mask, 
        #                                            index=true_class)[0]
    
    # return TransCAM_expln, LRP_expln, PartialLRP_expln, FullLRP_expln, Att_expln, Rollout_expln, Att_Gradcam_expln
    return TransCAM_expln, Att_expln, PartialLRP_expln, Rollout_expln, LRP_expln

In [15]:
def test1(model, explanations, test_data, degrade_step = 10, seg_ids=None):
    
    original_probs = []
    original_accs = [] 
    
    degradation_probs_TransCAM = []
    degradation_accs_TransCAM = []
    del_probs_TransCAM = []
    del_accs_TransCAM = []
    
    degradation_probs_Rawatt = []
    degradation_accs_Rawatt = []
    del_probs_Rawatt = []
    del_accs_Rawatt = []
    
        
    kendaltaus_TransCAM = []
    kendaltaus_Rawatt = []
    
    start_time = time.time()
    
    
    for i, test_instance in enumerate(test_data[1:10]):
        print("testinstant" + str(i) + "of" + str(len(test_data[1:10])))
        print("--- %s seconds ---" % (time.time() - start_time))
        text = test_instance['text']
        target = test_instance['label'] 

        text_ids, att_mask, text_words = preprocess_sample(text)
        
        # get truc words number
        total_len = len(text_words)
        if total_len< 10: 
            continue
        granularity = np.linspace(0, 1, degrade_step)
        trunc_words_num = [int(g) for g in np.round(granularity*total_len)]
        trunc_words_num = list(dict.fromkeys(trunc_words_num))
        
        original_class, original_prob = predict(model, text_ids, target)
        print("granularity: " +str(granularity)+ "trunc_words_num: " + str(trunc_words_num))
        
        # get attributions
        attribution_TransCAM, attribution_Rawatt, attribution_PartialLRP, attribution_Rollout, attribution_LRP = \
        generate_explns(explanations, text_ids, att_mask, start_layer=0, true_class = target, is_true = True) 
        
        attribution_TransCAM_F, attribution_Rawatt_F, attribution_PartialLRP_F, attribution_Rollout_F, attribution_LRP_F = \
        generate_explns(explanations, text_ids, att_mask, start_layer=0, true_class = target, is_true = False) 
               
        # conver to cpu numpy
        attribution_TransCAM = attribution_TransCAM.cpu().detach().numpy()
        attribution_Rawatt = attribution_Rawatt.cpu().detach().numpy()
        
        # conver to cpu numpy
        attribution_TransCAM_F = attribution_TransCAM_F.cpu().detach().numpy()
        attribution_Rawatt_F = attribution_Rawatt_F.cpu().detach().numpy()
 
        # get sorted_idx
        sorted_idx_TransCAM = np.argsort(-attribution_TransCAM)
        sorted_idx_Rawatt = np.argsort(-attribution_Rawatt)
        
        #degradation probabilities and accuracies:
        
        instance_degradation_probs_TransCAM = []
        instance_degradation_accs_TransCAM = []
        instance_replace_probs_TransCAM = []
        instance_replace_accs_TransCAM = []
        
        instance_degradation_probs_Rawatt = []
        instance_degradation_accs_Rawatt = []
        instance_replace_probs_Rawatt = []
        instance_replace_accs_Rawatt = []
        

        for num in trunc_words_num[1:]: #exclude 0
            
            # TransCAM
            truncated_text_ids_TransCAM, _, _ = truncate_words(sorted_idx_TransCAM, text_words, text_ids, 
                                                                                        num, seg_ids=seg_ids)
            replaced_text_ids_TransCAM, _ = replace_words(sorted_idx_TransCAM, text_words, text_ids, num, mask, mask_id)
           
            trunc_class_TransCAM, trunc_prob_TransCAM = predict(model, truncated_text_ids_TransCAM, target, seg_ids=seg_ids)
            rep_class_TransCAM, rep_prob_TransCAM = predict(model, replaced_text_ids_TransCAM, target, seg_ids=seg_ids)

            instance_degradation_probs_TransCAM.append(trunc_prob_TransCAM)
            instance_degradation_accs_TransCAM.append(trunc_class_TransCAM==target)
            
            instance_replace_probs_TransCAM.append(rep_prob_TransCAM)
            instance_replace_accs_TransCAM.append(rep_class_TransCAM==target)
            
            print(instance_degradation_probs_TransCAM, instance_degradation_accs_TransCAM)
            print(instance_replace_probs_TransCAM, instance_replace_accs_TransCAM)
            
            # Rawatt
            truncated_text_ids_Rawatt, _, _ = truncate_words(sorted_idx_Rawatt, text_words, text_ids, 
                                                                                       num, seg_ids=seg_ids)
            replaced_text_ids_Rawatt, _ = replace_words(sorted_idx_Rawatt, text_words, text_ids, num, mask, mask_id)
            
            
            trunc_class_Rawatt, trunc_prob_Rawatt = predict(model, truncated_text_ids_Rawatt, target, seg_ids=seg_ids)
            rep_class_Rawatt, rep_prob_Rawatt = predict(model, replaced_text_ids_Rawatt, target, seg_ids=seg_ids)

            instance_degradation_probs_Rawatt.append(trunc_prob_Rawatt)
            instance_degradation_accs_Rawatt.append(trunc_class_Rawatt==target)
            
            instance_replace_probs_Rawatt.append(rep_prob_Rawatt)
            instance_replace_accs_Rawatt.append(rep_class_Rawatt==target)
            
            
        original_probs.append(original_prob)
        original_accs.append(original_class==target)
        
        degradation_probs_TransCAM.append(instance_degradation_probs_TransCAM)
        degradation_accs_TransCAM.append(instance_degradation_accs_TransCAM)
        del_probs_TransCAM.append(instance_replace_probs_TransCAM)
        del_accs_TransCAM.append(instance_replace_accs_TransCAM)

        degradation_probs_Rawatt.append(instance_degradation_probs_Rawatt)
        degradation_accs_Rawatt.append(instance_degradation_accs_Rawatt)
        del_probs_Rawatt.append(instance_replace_probs_Rawatt)
        del_accs_Rawatt.append(instance_replace_accs_Rawatt)
        
        kendaltau_TransCAM = cal_kendaltau(attribution_TransCAM,attribution_TransCAM_F)
        kendaltaus_TransCAM.append(kendaltau_TransCAM)      
        
        kendaltau_Rawatt = cal_kendaltau(attribution_Rawatt,attribution_Rawatt_F)
        kendaltaus_Rawatt.append(kendaltau_Rawatt)      
        
 
    # get aopc score
    ## AOPC score for the "Rawatt" explanation method when words are deleted, while aopc_Rawatt represents the AOPC score for the "Rawatt" method when words are degraded or truncated.
    
    aopc_TransCAM = cal_aopc(original_probs,degradation_probs_TransCAM)
    aopc_Rawatt = cal_aopc(original_probs,degradation_probs_Rawatt)

    aopc_TransCAM_del = cal_aopc(original_probs,del_probs_TransCAM)
    aopc_Rawatt_del = cal_aopc(original_probs,del_probs_Rawatt)
     
        
    logodds_TransCAM = cal_logodds(original_probs,degradation_probs_TransCAM)
    logodds_Rawatt = cal_logodds(original_probs,degradation_probs_Rawatt)

    logodds_TransCAM_del = cal_logodds(original_probs,del_probs_TransCAM)
    logodds_Rawatt_del = cal_logodds(original_probs,del_probs_Rawatt)
      
    # get kendaltau score    
    k_TransCAM = np.mean(kendaltaus_TransCAM)
    k_Rawatt = np.mean(kendaltaus_Rawatt)

        
    return (aopc_TransCAM, aopc_Rawatt,
            aopc_TransCAM_del, aopc_Rawatt_del,
            k_TransCAM, k_Rawatt, 
            logodds_TransCAM, logodds_Rawatt,
            logodds_TransCAM_del, logodds_Rawatt_del)
    

In [ ]:
(aopc_TransCAM, aopc_Rawatt, 
 aopc_TransCAM_del, aopc_Rawatt_del, 
 k_TransCAM, k_Rawatt,
 logodds_TransCAM, logodds_Rawatt,
 logodds_TransCAM_del, logodds_Rawatt_del) = test1(model, explanations, to_test, degrade_step = 10, seg_ids=None)

In [ ]:
print(aopc_TransCAM, aopc_Rawatt, 
 aopc_TransCAM_del, aopc_Rawatt_del, 
 k_TransCAM, k_Rawatt,
 logodds_TransCAM, logodds_Rawatt,
 logodds_TransCAM_del, logodds_Rawatt_del)

In [ ]:
for j = length(original_prob)*0.9:
    0.58386946 0.2679196 0.09285318 0.11802776 0.019810585421362634 1.0 -4.249931055002206 -0.8265992346416297 0.7749939665525607 0.7417379722350244


for j = length(original_prob)*0.5:
    0.529297 0.17765257 0.07652367 0.111296386 0.019810585421362634 1.0 -3.535039852161189 -0.20997916727545052 0.6640482232641434 0.6113220509047019

for j = 9/ j = length(original_prob)*0.1: 
    aopc_TransCAM, aopc_Rawatt, 
 aopc_TransCAM_del, aopc_Rawatt_del, 
 k_TransCAM, k_Rawatt,
 logodds_TransCAM, logodds_Rawatt,
 logodds_TransCAM_del, logodds_Rawatt_del = 0.5953117 0.33234316 0.095037736 0.11741359 0.019810585421362634 1.0 -4.0508902344676825 -0.9658935394674223 0.7876196518802633 0.758062799867656
    

In [16]:
def test(model, explanations, test_data, degrade_step = 10, seg_ids=None):
    
    original_probs = []
    original_accs = [] 
    
    degradation_probs_TransCAM = []
    degradation_accs_TransCAM = []
    del_probs_TransCAM = []
    del_accs_TransCAM = []
    
    degradation_probs_Rawatt = []
    degradation_accs_Rawatt = []
    del_probs_Rawatt = []
    del_accs_Rawatt = []
    
    degradation_probs_PartialLRP = []
    degradation_accs_PartialLRP = []
    del_probs_PartialLRP = []
    del_accs_PartialLRP = []
    
    degradation_probs_Rollout = []
    degradation_accs_Rollout = []
    del_probs_Rollout = []
    del_accs_Rollout = []
    
    degradation_probs_LRP = []
    degradation_accs_LRP = []
    del_probs_LRP = []
    del_accs_LRP = []
    
    kendaltaus_TransCAM = []
    kendaltaus_Rawatt = []
    kendaltaus_PartialLRP = []
    kendaltaus_Rollout = []
    kendaltaus_LRP = []
    
    start_time = time.time()
    
    
    for i, test_instance in enumerate(test_data):
        if i%100 == 0:
            print("testinstant" + str(i) + "of" + str(len(test_data)))
            print("--- %s seconds ---" % (time.time() - start_time))
        
        text = test_instance['text']
        target = test_instance['label'] 

        text_ids, att_mask, text_words = preprocess_sample(text)
        
        # get truc words number
        total_len = len(text_words)
        if total_len< 10: 
            continue
            
        #trunc_words_num created using granularity which represents percentage of words to be masked
        #calculated based on total length of text_words
        
        granularity = np.linspace(0, 1, degrade_step)
        trunc_words_num = [int(g) for g in np.round(granularity*total_len)]
        trunc_words_num = list(dict.fromkeys(trunc_words_num))
        
        original_class, original_prob = predict(model, text_ids, target)
        # get attributions
        attribution_TransCAM, attribution_Rawatt, attribution_PartialLRP, attribution_Rollout, attribution_LRP = \
        generate_explns(explanations, text_ids, att_mask, start_layer=0, true_class = target, is_true = True) 
        
        attribution_TransCAM_F, attribution_Rawatt_F, attribution_PartialLRP_F, attribution_Rollout_F, attribution_LRP_F = \
        generate_explns(explanations, text_ids, att_mask, start_layer=0, true_class = target, is_true = False) 
        
        # conver to cpu numpy
        attribution_TransCAM = attribution_TransCAM.cpu().detach().numpy()
        attribution_Rawatt = attribution_Rawatt.cpu().detach().numpy()
        attribution_PartialLRP = attribution_PartialLRP.cpu().detach().numpy()
        attribution_Rollout = attribution_Rollout.cpu().detach().numpy()
        attribution_LRP = attribution_LRP.cpu().detach().numpy()
        
        # conver to cpu numpy
        attribution_TransCAM_F = attribution_TransCAM_F.cpu().detach().numpy()
        attribution_Rawatt_F = attribution_Rawatt_F.cpu().detach().numpy()
        attribution_PartialLRP_F = attribution_PartialLRP_F.cpu().detach().numpy()
        attribution_Rollout_F = attribution_Rollout_F.cpu().detach().numpy()
        attribution_LRP_F = attribution_LRP_F.cpu().detach().numpy()
        
        # get sorted_idx
        sorted_idx_TransCAM = np.argsort(-attribution_TransCAM)
        sorted_idx_Rawatt = np.argsort(-attribution_Rawatt)
        sorted_idx_PartialLRP = np.argsort(-attribution_PartialLRP)
        sorted_idx_Rollout = np.argsort(-attribution_Rollout)
        sorted_idx_LRP = np.argsort(-attribution_LRP)
        
        instance_degradation_probs_TransCAM = []
        instance_degradation_accs_TransCAM = []
        instance_replace_probs_TransCAM = []
        instance_replace_accs_TransCAM = []
        
        instance_degradation_probs_Rawatt = []
        instance_degradation_accs_Rawatt = []
        instance_replace_probs_Rawatt = []
        instance_replace_accs_Rawatt = []
        
        instance_degradation_probs_PartialLRP = []
        instance_degradation_accs_PartialLRP = []
        instance_replace_probs_PartialLRP = []
        instance_replace_accs_PartialLRP = []
        
        instance_degradation_probs_Rollout = []
        instance_degradation_accs_Rollout = []
        instance_replace_probs_Rollout = []
        instance_replace_accs_Rollout = []
        
        instance_degradation_probs_LRP = []
        instance_degradation_accs_LRP = []
        instance_replace_probs_LRP = []
        instance_replace_accs_LRP = []

        for num in trunc_words_num[1:]: #exclude 0
         #num represents the total number of words to be masked. trunc_words_num is sorted list of number of words to be masked at each step -> loop iterates over values
            
            # TransCAM
            truncated_text_ids_TransCAM, _, _ = truncate_words(sorted_idx_TransCAM, text_words, text_ids, 
                                                                                        num, seg_ids=seg_ids)
            replaced_text_ids_TransCAM, _ = replace_words(sorted_idx_TransCAM, text_words, text_ids, num, mask, mask_id)
           
            trunc_class_TransCAM, trunc_prob_TransCAM = predict(model, truncated_text_ids_TransCAM, target, seg_ids=seg_ids)
            rep_class_TransCAM, rep_prob_TransCAM = predict(model, replaced_text_ids_TransCAM, target, seg_ids=seg_ids)

            instance_degradation_probs_TransCAM.append(trunc_prob_TransCAM)
            instance_degradation_accs_TransCAM.append(trunc_class_TransCAM==target)
            
            instance_replace_probs_TransCAM.append(rep_prob_TransCAM)
            instance_replace_accs_TransCAM.append(rep_class_TransCAM==target)
            
            # Rawatt
            truncated_text_ids_Rawatt, _, _ = truncate_words(sorted_idx_Rawatt, text_words, text_ids, 
                                                                                       num, seg_ids=seg_ids)
            replaced_text_ids_Rawatt, _ = replace_words(sorted_idx_Rawatt, text_words, text_ids, num, mask, mask_id)
            
            
            trunc_class_Rawatt, trunc_prob_Rawatt = predict(model, truncated_text_ids_Rawatt, target, seg_ids=seg_ids)
            rep_class_Rawatt, rep_prob_Rawatt = predict(model, replaced_text_ids_Rawatt, target, seg_ids=seg_ids)

            instance_degradation_probs_Rawatt.append(trunc_prob_Rawatt)
            instance_degradation_accs_Rawatt.append(trunc_class_Rawatt==target)
            
            instance_replace_probs_Rawatt.append(rep_prob_Rawatt)
            instance_replace_accs_Rawatt.append(rep_class_Rawatt==target)
            
            # PartialLRP
            truncated_text_ids_PartialLRP, _, _ = truncate_words(sorted_idx_PartialLRP, text_words, text_ids, 
                                                                                       num, seg_ids=seg_ids)
            replaced_text_ids_PartialLRP, _ = replace_words(sorted_idx_PartialLRP, text_words, text_ids, num, mask, mask_id)
            
            trunc_class_PartialLRP, trunc_prob_PartialLRP = predict(model, truncated_text_ids_PartialLRP, target, seg_ids=seg_ids)
            rep_class_PartialLRP, rep_prob_PartialLRP = predict(model, replaced_text_ids_PartialLRP, target, seg_ids=seg_ids)
            
            instance_degradation_probs_PartialLRP.append(trunc_prob_PartialLRP)
            instance_degradation_accs_PartialLRP.append(trunc_class_PartialLRP==target)
            
            instance_replace_probs_PartialLRP.append(rep_prob_PartialLRP)
            instance_replace_accs_PartialLRP.append(rep_class_PartialLRP==target)
            
            # Rollout
            truncated_text_ids_Rollout, _, _ = truncate_words(sorted_idx_Rollout, text_words, text_ids, 
                                                                                       num, seg_ids=seg_ids)
            replaced_text_ids_Rollout, _ = replace_words(sorted_idx_Rollout, text_words, text_ids, num, mask, mask_id)
            
            trunc_class_Rollout, trunc_prob_Rollout = predict(model, truncated_text_ids_Rollout, target, seg_ids=seg_ids)
            rep_class_Rollout, rep_prob_Rollout = predict(model, replaced_text_ids_Rollout, target, seg_ids=seg_ids)
            
            instance_degradation_probs_Rollout.append(trunc_prob_Rollout)
            instance_degradation_accs_Rollout.append(trunc_class_Rollout==target)
            
            instance_replace_probs_Rollout.append(rep_prob_Rollout)
            instance_replace_accs_Rollout.append(rep_class_Rollout==target)
            
            # LRP
            truncated_text_ids_LRP, _, _ = truncate_words(sorted_idx_LRP, text_words, text_ids, 
                                                                                       num, seg_ids=seg_ids)
            replaced_text_ids_LRP, _ = replace_words(sorted_idx_LRP, text_words, text_ids, num, mask, mask_id)
                
            trunc_class_LRP, trunc_prob_LRP = predict(model, truncated_text_ids_LRP, target, seg_ids=seg_ids)
            rep_class_LRP, rep_prob_LRP = predict(model, replaced_text_ids_LRP, target, seg_ids=seg_ids)

            instance_degradation_probs_LRP.append(trunc_prob_LRP)
            instance_degradation_accs_LRP.append(trunc_class_LRP==target)
            
            instance_replace_probs_LRP.append(rep_prob_LRP)
            instance_replace_accs_LRP.append(rep_class_LRP==target)

        original_probs.append(original_prob)
        original_accs.append(original_class==target)
        
        degradation_probs_TransCAM.append(instance_degradation_probs_TransCAM)
        degradation_accs_TransCAM.append(instance_degradation_accs_TransCAM)
        del_probs_TransCAM.append(instance_replace_probs_TransCAM)
        del_accs_TransCAM.append(instance_replace_accs_TransCAM)
        
        
        degradation_probs_Rawatt.append(instance_degradation_probs_Rawatt)
        degradation_accs_Rawatt.append(instance_degradation_accs_Rawatt)
        del_probs_Rawatt.append(instance_replace_probs_Rawatt)
        del_accs_Rawatt.append(instance_replace_accs_Rawatt)
        
        degradation_probs_PartialLRP.append(instance_degradation_probs_PartialLRP)
        degradation_accs_PartialLRP.append(instance_degradation_accs_PartialLRP)
        del_probs_PartialLRP.append(instance_replace_probs_PartialLRP)
        del_accs_PartialLRP.append(instance_replace_accs_PartialLRP)
        
        degradation_probs_Rollout.append(instance_degradation_probs_Rollout)
        degradation_accs_Rollout.append(instance_degradation_accs_Rollout)
        del_probs_Rollout.append(instance_replace_probs_Rollout)
        del_accs_Rollout.append(instance_replace_accs_Rollout)
        
        degradation_probs_LRP.append(instance_degradation_probs_LRP)
        degradation_accs_LRP.append(instance_degradation_accs_LRP)
        del_probs_LRP.append(instance_replace_probs_LRP)
        del_accs_LRP.append(instance_replace_accs_LRP)
        
        
        kendaltau_TransCAM = cal_kendaltau(attribution_TransCAM,attribution_TransCAM_F)
        kendaltaus_TransCAM.append(kendaltau_TransCAM)      
        
        kendaltau_Rawatt = cal_kendaltau(attribution_Rawatt,attribution_Rawatt_F)
        kendaltaus_Rawatt.append(kendaltau_Rawatt)      
        
        kendaltau_PartialLRP = cal_kendaltau(attribution_PartialLRP,attribution_PartialLRP_F)
        kendaltaus_PartialLRP.append(kendaltau_PartialLRP)      
        
        kendaltau_Rollout = cal_kendaltau(attribution_Rollout,attribution_Rollout_F)
        kendaltaus_Rollout.append(kendaltau_Rollout)      
        
        kendaltau_LRP = cal_kendaltau(attribution_LRP,attribution_LRP_F)
        kendaltaus_LRP.append(kendaltau_LRP)      
        
    # get aopc score
    aopc_TransCAM = cal_aopc(original_probs,degradation_probs_TransCAM)
    aopc_Rawatt = cal_aopc(original_probs,degradation_probs_Rawatt)
    aopc_PartialLRP = cal_aopc(original_probs,degradation_probs_PartialLRP)
    aopc_Rollout = cal_aopc(original_probs,degradation_probs_Rollout)
    aopc_LRP = cal_aopc(original_probs,degradation_probs_LRP)
    
    aopc_TransCAM_del = cal_aopc(original_probs,del_probs_TransCAM)
    aopc_Rawatt_del = cal_aopc(original_probs,del_probs_Rawatt)
    aopc_PartialLRP_del = cal_aopc(original_probs,del_probs_PartialLRP)
    aopc_Rollout_del = cal_aopc(original_probs,del_probs_Rollout)
    aopc_LRP_del = cal_aopc(original_probs,del_probs_LRP)
        
        
    logodds_TransCAM = cal_logodds(original_probs,degradation_probs_TransCAM)
    logodds_Rawatt = cal_logodds(original_probs,degradation_probs_Rawatt)
    logodds_PartialLRP = cal_logodds(original_probs,degradation_probs_PartialLRP)
    logodds_Rollout = cal_logodds(original_probs,degradation_probs_Rollout)
    logodds_LRP = cal_logodds(original_probs,degradation_probs_LRP)
    
    logodds_TransCAM_del = cal_logodds(original_probs,del_probs_TransCAM)
    logodds_Rawatt_del = cal_logodds(original_probs,del_probs_Rawatt)
    logodds_PartialLRP_del = cal_logodds(original_probs,del_probs_PartialLRP)
    logodds_Rollout_del = cal_logodds(original_probs,del_probs_Rollout)
    logodds_LRP_del = cal_logodds(original_probs,del_probs_LRP)
        
            
    # get kendaltau score    
    k_TransCAM = np.mean(kendaltaus_TransCAM)
    k_Rawatt = np.mean(kendaltaus_Rawatt)
    k_PartialLRP = np.mean(kendaltaus_PartialLRP)
    k_Rollout = np.mean(kendaltaus_Rollout)
    k_LRP = np.mean(kendaltaus_LRP)

        
    return (aopc_TransCAM, aopc_Rawatt, aopc_PartialLRP, aopc_Rollout, aopc_LRP,
            aopc_TransCAM_del, aopc_Rawatt_del, aopc_PartialLRP_del, aopc_Rollout_del, aopc_LRP_del,
            k_TransCAM, k_Rawatt, k_PartialLRP, k_Rollout, k_LRP,
            logodds_TransCAM, logodds_Rawatt, logodds_PartialLRP, logodds_Rollout, logodds_LRP,
            logodds_TransCAM_del, logodds_Rawatt_del, logodds_PartialLRP_del, logodds_Rollout_del, logodds_LRP_del)
    

In [17]:
def cal_aopc(original_probs, degradation_probs):
    original_probs = np.array(original_probs)
    degradation_probs = np.array(degradation_probs)
    
    diffs = []
    for i in range(len(original_probs)):
        diffs_k = []
        for j in range(9):
            diff = original_probs[i] - degradation_probs[i][j]
            diffs_k.append(np.abs(diff))
        diffs.append(diffs_k)
  
    result = np.mean(diffs, axis=0)
    aopc = np.mean(result)
    
    return aopc

def cal_logodds(original_probs, degradation_probs):
    original_probs = np.array(original_probs)
    degradation_probs = np.array(degradation_probs)
    
    ratios = []
    for i in range(len(original_probs)):
        ratios_k = []
        for j in range(9):
            ratio = math.log(degradation_probs[i][j] / original_probs[i])
            ratios_k.append(ratio)
        ratios.append(ratios_k)

    result = np.mean(ratios, axis=0)
    logodds = np.mean(result)
    
    return logodds

def cal_kendaltau(attribution1, attribution2,):

    sorted_idx1 = np.argsort(-attribution1)
    sorted_idx2 = np.argsort(-attribution2)

    tau, p_value = stats.kendalltau(sorted_idx1, sorted_idx2)
    
    return tau

In [18]:
# initialize the explanations generator
explanations = Generator(model)

In [19]:
# test_num = 100
to_test = np.array(dataset['test'])
# to_test_idx = np.random.choice(len(dataset['test']), test_num, replace=False)
# to_test = to_test[to_test_idx]
# print(len(to_test))

In [23]:
(aopc_TransCAM, aopc_Rawatt, aopc_PartialLRP, aopc_Rollout, aopc_LRP,
 aopc_TransCAM_del, aopc_Rawatt_del, aopc_PartialLRP_del, aopc_Rollout_del, aopc_LRP_del,
 k_TransCAM, k_Rawatt, k_PartialLRP, k_Rollout, k_LRP,
 logodds_TransCAM, logodds_Rawatt, logodds_PartialLRP, logodds_Rollout, logodds_LRP,
 logodds_TransCAM_del, logodds_Rawatt_del, logodds_PartialLRP_del, logodds_Rollout_del, logodds_LRP_del) = test(model, explanations, to_test, degrade_step = 10, seg_ids=None)

testinstant0of25000
--- 0.0001430511474609375 seconds ---


/home/anwbw/.local/lib/python3.9/site-packages/transformers/modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


testinstant100of25000
--- 387.5907051563263 seconds ---
testinstant200of25000
--- 744.6323065757751 seconds ---
testinstant300of25000
--- 1104.391256570816 seconds ---
testinstant400of25000
--- 1495.6223006248474 seconds ---
testinstant500of25000
--- 1874.7481126785278 seconds ---
testinstant600of25000
--- 2251.646807193756 seconds ---
testinstant700of25000
--- 2606.659215927124 seconds ---
testinstant800of25000
--- 2973.3355662822723 seconds ---
testinstant900of25000
--- 3324.9155955314636 seconds ---
testinstant1000of25000
--- 3710.8555102348328 seconds ---
testinstant1100of25000
--- 4056.4538235664368 seconds ---
testinstant1200of25000
--- 4434.036741256714 seconds ---
testinstant1300of25000
--- 4825.678437948227 seconds ---
testinstant1400of25000
--- 5200.99964094162 seconds ---
testinstant1500of25000
--- 5529.142740011215 seconds ---
testinstant1600of25000
--- 5882.350497484207 seconds ---
testinstant1700of25000
--- 6236.152051210403 seconds ---
testinstant1800of25000
--- 6585.388

testinstant14500of25000
--- 53636.69331598282 seconds ---
testinstant14600of25000
--- 54025.159907102585 seconds ---
testinstant14700of25000
--- 54378.50980305672 seconds ---
testinstant14800of25000
--- 54750.84029555321 seconds ---
testinstant14900of25000
--- 55104.7652554512 seconds ---
testinstant15000of25000
--- 55459.24174451828 seconds ---
testinstant15100of25000
--- 55837.54755830765 seconds ---
testinstant15200of25000
--- 56211.51493906975 seconds ---
testinstant15300of25000
--- 56579.74733543396 seconds ---
testinstant15400of25000
--- 56963.255464076996 seconds ---
testinstant15500of25000
--- 57330.95045185089 seconds ---
testinstant15600of25000
--- 57658.89557361603 seconds ---
testinstant15700of25000
--- 58051.63272356987 seconds ---
testinstant15800of25000
--- 58407.422463178635 seconds ---
testinstant15900of25000
--- 58735.7085750103 seconds ---
testinstant16000of25000
--- 59123.6877245903 seconds ---
testinstant16100of25000
--- 59468.36649775505 seconds ---
testinstant162

In [24]:
# GPU information
import GPUtil
from tabulate import tabulate
print("="*40, "GPU Details", "="*40)
gpus = GPUtil.getGPUs()
list_gpus = []
for gpu in gpus:
    # get the GPU id
    gpu_id = gpu.id
    # name of GPU
    gpu_name = gpu.name
    # get % percentage of GPU usage of that GPU
    gpu_load = f"{gpu.load*100}%"
    # get free memory in MB format
    gpu_free_memory = f"{gpu.memoryFree}MB"
    # get used memory
    gpu_used_memory = f"{gpu.memoryUsed}MB"
    # get total memory
    gpu_total_memory = f"{gpu.memoryTotal}MB"
    # get GPU temperature in Celsius
    gpu_temperature = f"{gpu.temperature} °C"
    gpu_uuid = gpu.uuid
    list_gpus.append((
        gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
        gpu_total_memory, gpu_temperature, gpu_uuid
    ))

print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))

======================================== GPU Details ========================================
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  id  name          load    free memory    used memory    total memory    temperature    uuid
----  ------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Quadro P4000  0.0%    1354.0MB       6737.0MB       8192.0MB        70.0 °C        GPU-d54fc58e-a862-524b-2c49-5328a9082994


In [25]:
 print(aopc_TransCAM,aopc_Rawatt,aopc_PartialLRP,aopc_Rollout,aopc_LRP)
 print(aopc_TransCAM_del,aopc_Rawatt_del,aopc_PartialLRP_del,aopc_Rollout_del,aopc_LRP_del)
 print(k_TransCAM,k_Rawatt,k_PartialLRP,k_Rollout,k_LRP)
 print(logodds_TransCAM, logodds_Rawatt, logodds_PartialLRP, logodds_Rollout, logodds_LRP)
 print(logodds_TransCAM_del, logodds_Rawatt_del, logodds_PartialLRP_del, logodds_Rollout_del, logodds_LRP_del)

0.5266037 0.31141058 0.33650267 0.3204375 0.39737538
0.4478191 0.47630623 0.4819205 0.4777875 0.49614155
0.02527974941758765 1.0 0.8598853238499905 1.0 0.05708033662736514
-3.6611068678825953 -1.2512246670921452 -1.381179370802607 -1.2706752994204191 -1.9169275222101259
-2.6011232209657456 -2.7190599917127893 -2.7631337260930477 -2.7876535090996586 -2.8977645609038993


In [23]:
aopc_TransCAM,aopc_Rawatt,aopc_PartialLRP,aopc_Rollout,aopc_LRP = [0.5266037, 0.31141058, 0.33650267, 0.3204375, 0.39737538]
aopc_TransCAM_del,aopc_Rawatt_del,aopc_PartialLRP_del,aopc_Rollout_del,aopc_LRP_del =[0.4478191, 0.47630623, 0.4819205, 0.4777875, 0.49614155]
k_TransCAM,k_Rawatt,k_PartialLRP,k_Rollout,k_LRP = [0.02527974941758765, 1.0, 0.8598853238499905, 1.0, 0.05708033662736514]
logodds_TransCAM, logodds_Rawatt, logodds_PartialLRP, logodds_Rollout, logodds_LRP = [-3.6611068678825953, -1.2512246670921452, -1.381179370802607, -1.2706752994204191, -1.9169275222101259]
logodds_TransCAM_del, logodds_Rawatt_del, logodds_PartialLRP_del, logodds_Rollout_del, logodds_LRP_del = [-2.6011232209657456, -2.7190599917127893, -2.7631337260930477, -2.7876535090996586, -2.8977645609038993]

# Single Example Visualization

In [67]:
def expln_norm(expln):
    expln = (expln - expln.min()) / (expln.max()- expln.min())
    return expln

def show_text_attr(expln,str_list,is_relu = True):
    if is_relu:
        rgb = lambda x: '0,0,0' if x < 0 else '0,255,0'
        alpha = lambda x: max(x, 0)# ** 0.5
    else:
        rgb = lambda x: '255,0,0' if x < 0 else '0,255,0'
        alpha = lambda x: x * -5 if x < 0 else x * 100
    attrs = list(expln)
    subwords = str_list
    
    token_marks = [
        f'<mark style="background-color:rgba({rgb(attr)},{alpha(attr)})">{token}</mark>'
        for token, attr in zip(subwords, attrs)
    ]
    
    display(HTML('<p>' + ' '.join(token_marks) + '</p>'))


In [68]:
text_ids, att_mask, text_words = preprocess_sample(to_test[3]['text'])
label = to_test[3]['label'] 

In [69]:
#TransCAM_expln, LRP_expln, PartialLRP_expln, FullLRP_expln, Att_expln, Rollout_expln, Att_Gradcam_expln = \
        #generate_explns(explanations, text_ids, att_mask, start_layer=0, true_class = label, is_true = True) 
    
TransCAM_expln, LRP_expln, PartialLRP_expln, Att_expln, Rollout_expln = \
        generate_explns(explanations, text_ids, att_mask, start_layer=0, true_class = label, is_true = True)

In [70]:
print(TransCAM_expln)

tensor([ 0.0000e+00,  2.0246e-05, -9.0851e-05,  5.3050e-06,  5.2688e-06,
        -6.5191e-06, -2.6757e-06, -2.7638e-06, -1.9164e-06,  6.3683e-05,
         4.7194e-05, -7.4795e-06, -4.5081e-06, -1.3562e-06, -1.8982e-07,
         4.2286e-05,  4.1705e-05,  8.3080e-06, -1.2340e-06, -1.3859e-06,
         3.9276e-05, -6.2516e-06, -1.8857e-06, -1.2346e-06,  8.2779e-06,
         3.1831e-06, -1.4231e-05,  3.8019e-05,  3.7485e-07,  9.2244e-06,
         2.1775e-06,  7.5284e-06, -1.5060e-06,  2.1868e-06, -4.2077e-06,
         1.4373e-06, -7.4659e-06, -5.8707e-06,  3.5607e-06, -1.9910e-04,
         2.3172e-05, -1.4256e-05, -2.0419e-06,  1.9726e-05, -5.5410e-06,
        -8.1794e-06, -4.9918e-05, -1.2873e-05,  7.5615e-06, -6.4310e-06,
         4.3131e-06,  4.4779e-06,  1.2914e-04,  2.6213e-05, -1.7414e-05,
         6.4731e-05,  1.3628e-05, -3.1390e-06,  9.2370e-06, -8.5702e-06,
        -1.1848e-06,  4.7681e-06, -4.5346e-06, -4.0183e-05, -1.5487e-05,
         2.8711e-06, -6.5494e-06,  2.5351e-06,  1.7

In [71]:
#TransCAM_expln = expln_norm(TransCAM_expln)
show_text_attr(TransCAM_expln,text_words,is_relu = False)

In [40]:
 #LRP_expln = expln_norm(LRP_expln)
 show_text_attr(LRP_expln,text_words,is_relu = False)

In [43]:
PartialLRP_expln = expln_norm(PartialLRP_expln)
show_text_attr(PartialLRP_expln,text_words,is_relu = False)

In [41]:
#Att_expln = expln_norm(Att_expln)
show_text_attr(Att_expln,text_words,is_relu = False)

In [57]:
Rollout_expln = expln_norm(Rollout_expln)
show_text_attr(Rollout_expln,text_words,is_relu = False)